## This notebook provides a simple comparison of computation time of the python version and the new Cython/C++ version.

### Importing libraries

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import fif as FIF
import old_fif as old_FIF

### Simulated functions

In [5]:
np.random.seed(42)
m =1000 ## m measures
n =1000 ## n curves
tps = np.linspace(0,1,m)
v = np.linspace(1,1.4,n)
X = np.zeros((n,m))
for i in range(n):
    X[i] = 30 * ((1-tps) ** v[i]) * tps ** v[i]


Z1 = np.zeros((m))
for j in range(m):
    if (tps[j]<0.2 or tps[j]>0.8):
        Z1[j] = 30 * ((1-tps[j]) ** 1.2) * tps[j] ** 1.2
    else:
        Z1[j] = 30 * ((1-tps[j]) ** 1.2) * tps[j] ** 1.2 + np.random.normal(0,0.3,1)
Z1[0] = 0
Z1[m-1] = 0


Z2 = 30 * ((1-tps) ** 1.6) * tps ** 1.6


Z3 = np.zeros((m))
for j in range(m):
    Z3[j] = 30 * ((1-tps[j]) ** 1.2) * tps[j] ** 1.2 + np.sin(2*np.pi*tps[j])

Z4 = np.zeros((m))
for j in range(m):
    Z4[j] = 30 * ((1-tps[j]) ** 1.2) * tps[j] ** 1.2

for j in range(70,71):
    Z4[j] += 2

Z5 = np.zeros((m))
for j in range(m):
    Z5[j] = 30 * ((1-tps[j]) ** 1.2) * tps[j] ** 1.2 + 0.5*np.sin(10*np.pi*tps[j])

X = np.concatenate((X,Z1.reshape(1,-1),Z2.reshape(1,-1),
                     Z3.reshape(1,-1), Z4.reshape(1,-1), Z5.reshape(1,-1)), axis = 0)

## Cython/C++ version of FIF

In [8]:
%%time
time = np.linspace(0,1,m)
F = FIF.FiForest(X, time, ntrees=300, sample_size=64, alpha=0.5, dic_number=1, seed=0)
S_new_F = F.compute_paths(X_in=X)

CPU times: user 7.21 s, sys: 175 ms, total: 7.38 s
Wall time: 7.33 s


## Old python version of FIF

In [9]:
%%time 
np.random.seed(0)
old_F = old_FIF.FIForest(X,ntrees=300,subsample_size=64,time=time, D='gaussian_wavelets', innerproduct='auto', alpha=0.5)
S_old_F = old_F.compute_paths(X)

CPU times: user 5min 48s, sys: 595 ms, total: 5min 48s
Wall time: 5min 49s
